# INST414 Final Project Sprint 3

In [18]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datascience import *
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import scipy.stats as stats
from statsmodels.stats.outliers_influence import variance_inflation_factor

## Data Cleaning

In [19]:
# select columns to read and keep
cols_to_keep = ['Data_Value', 'Race/Ethnicity', 'Age(months)', 'Sex', 'LocationDesc', 'LocationAbbr', 'YearStart', 'YearEnd', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size', 'GeoLocation']
df = pd.read_csv("/Users/virginialee/Downloads/WIC_data.csv", usecols=cols_to_keep)

df.head()
df.shape

(12852, 12)

In [20]:
# rename columns to be more intuitive
df.rename(columns={
    'Data_Value': 'pct_overweight',
    'Race/Ethnicity': 'race',
    'Age(months)': 'age_months'
}, inplace=True)
df.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,Sex,race,GeoLocation
0,2008,2008,AL,Alabama,15.3,14.7,15.8,"18,219",24 - 35,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
1,2008,2008,AL,Alabama,14.9,14.4,15.5,"14,796",36 - 47,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
2,2008,2008,AL,Alabama,16.4,15.6,17.1,"10,272",48 - 59,NaN,NaN,"(32.84057112200048, -86.63186076199969)"
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,NaN,American Indian/Alaska Native,"(32.84057112200048, -86.63186076199969)"
4,2008,2008,AL,Alabama,8.8,5.4,12.2,273,NaN,NaN,Asian/Pacific Islander,"(32.84057112200048, -86.63186076199969)"


In [21]:
# checking missing values
missing_value = df.isna().sum()
print(missing_value)

YearStart                    0
YearEnd                      0
LocationAbbr                 0
LocationDesc                 0
pct_overweight             379
Low_Confidence_Limit       379
High_Confidence_Limit      379
Sample_Size                379
age_months                9072
Sex                      10584
race                      7182
GeoLocation                  0
dtype: int64


In [165]:
# calculate % missing for key columns
379/12852 # 2.9% missing for data_value
379/12852 # 2.9% missing for low_confidence_limit
379/12852 # 2.9% missing for high_confidence_limit
379/12852 # 2.9% missing for sample_size
7182/12852 # 55.88% missing for race/ethnicity
9072/12852 # 70.59% missing for age
10584/12852 # 82.35% missing for sex

0.8235294117647058

In [22]:
# dropping missing values and unnecessary columns
clean_df = df.dropna(subset=['pct_overweight', 'Low_Confidence_Limit', 'High_Confidence_Limit', 'Sample_Size'])
clean_df = clean_df.drop(columns=['Sex'])
clean_df.head()
clean_df.shape

(12473, 11)

In [23]:
# filter race by American Indian/Alaska Native and Non-Hispanic White
df_race = clean_df[clean_df['race'].isin(['American Indian/Alaska Native', 'Non-Hispanic White'])]
df_race.shape
df_race.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,American Indian/Alaska Native,"(32.84057112200048, -86.63186076199969)"
9,2008,2008,AL,Alabama,15.8,15.2,16.3,"17,833",NaN,Non-Hispanic White,"(32.84057112200048, -86.63186076199969)"
14,2008,2008,AK,Alaska,23.7,22.2,25.2,"3,190",NaN,American Indian/Alaska Native,"(64.84507995700051, -147.72205903599973)"
20,2008,2008,AK,Alaska,15.5,14.3,16.7,"3,540",NaN,Non-Hispanic White,"(64.84507995700051, -147.72205903599973)"
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,"1,101",NaN,American Indian/Alaska Native,"(34.865970280000454, -111.76381127699972)"


In [24]:
# duplicates in data set
print(df_race.duplicated().value_counts()) # how many rows are duplicates
df_race[df_race.duplicated(keep=False)] # displays duplicate rows
# drop duplicates
df_race = df_race.drop_duplicates()
df_race.shape

False    2021
True        8
Name: count, dtype: int64


(2021, 11)

In [25]:
# relabel values to be more intuitive
df_race = df_race.replace({
    'American Indian/Alaska Native': 'Indigenous',
    'Non-Hispanic White': 'White'
}, inplace=False)
df_race.head()

/var/folders/19/jdc_mhnj0kb5ql3ygtht4_0m0000gn/T/ipykernel_18782/1362604716.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_race = df_race.replace({


,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)"
9,2008,2008,AL,Alabama,15.8,15.2,16.3,"17,833",NaN,White,"(32.84057112200048, -86.63186076199969)"
14,2008,2008,AK,Alaska,23.7,22.2,25.2,"3,190",NaN,Indigenous,"(64.84507995700051, -147.72205903599973)"
20,2008,2008,AK,Alaska,15.5,14.3,16.7,"3,540",NaN,White,"(64.84507995700051, -147.72205903599973)"
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,"1,101",NaN,Indigenous,"(34.865970280000454, -111.76381127699972)"


In [26]:
# turn race into boolean variable
df_race.drop(columns='Indigenous', errors='ignore', inplace=True)  # drop if already exists
df_race['Indigenous'] = (df_race['race'] == 'Indigenous').astype(int)
df_race.head()
df_race.to_csv('cleaned_wic_data.csv', index=False)

## Baseline Linear Regression Model

In [27]:
# Fit the linear regression model using sklearn
X = df_race[['Indigenous']]
y = df_race['pct_overweight']
reg = LinearRegression()
reg.fit(X, y)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [28]:
reg.coef_  # coefficient for Indigenous variable
reg.intercept_  # intercept
reg.score(X, y)  # R-squared value
print("Coefficient for Indigenous:", reg.coef_[0], "Intercept:", reg.intercept_, "R-squared:", reg.score(X, y))

Coefficient for Indigenous: 3.72293355788 Intercept: 12.905380334 R-squared: 0.16973356136433138


In [29]:
# using statsmodels to get p-value
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.170
Model:                            OLS   Adj. R-squared:                  0.169
Method:                 Least Squares   F-statistic:                     412.7
Date:                Sun, 14 Dec 2025   Prob (F-statistic):           1.21e-83
Time:                        17:01:14   Log-Likelihood:                -5723.1
No. Observations:                2021   AIC:                         1.145e+04
Df Residuals:                    2019   BIC:                         1.146e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.9054      0.125    103.099      0.0

## Primary Model

In [31]:
# writing a function to remove commas for Sample_Size column
def remove_commas(df, column_name):
    df[column_name] = (df[column_name].astype(str).str.replace(',', '', regex=False).astype(int))

# testing function on a copy of df_race
test_fx = df_race.copy()

remove_commas(test_fx, 'Sample_Size')
test_fx.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation,Indigenous
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)",1
9,2008,2008,AL,Alabama,15.8,15.2,16.3,17833,NaN,White,"(32.84057112200048, -86.63186076199969)",0
14,2008,2008,AK,Alaska,23.7,22.2,25.2,3190,NaN,Indigenous,"(64.84507995700051, -147.72205903599973)",1
20,2008,2008,AK,Alaska,15.5,14.3,16.7,3540,NaN,White,"(64.84507995700051, -147.72205903599973)",0
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,1101,NaN,Indigenous,"(34.865970280000454, -111.76381127699972)",1


In [33]:
# applying function to df_race
remove_commas(df_race, 'Sample_Size')
df_race.head()

,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation,Indigenous
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)",1
9,2008,2008,AL,Alabama,15.8,15.2,16.3,17833,NaN,White,"(32.84057112200048, -86.63186076199969)",0
14,2008,2008,AK,Alaska,23.7,22.2,25.2,3190,NaN,Indigenous,"(64.84507995700051, -147.72205903599973)",1
20,2008,2008,AK,Alaska,15.5,14.3,16.7,3540,NaN,White,"(64.84507995700051, -147.72205903599973)",0
25,2008,2008,AZ,Arizona,20.1,17.7,22.4,1101,NaN,Indigenous,"(34.865970280000454, -111.76381127699972)",1


In [34]:
# Independent Variables
# converting Sample_Size and pct_overweight to numeric for statsmodel
df_race['Sample_Size'] = pd.to_numeric(df_race['Sample_Size'], errors='coerce')
df_race['pct_overweight'] = pd.to_numeric(df_race['pct_overweight'], errors='coerce')

X = df_race[['Indigenous']]
X = sm.add_constant(X)

# Dependent Variables
y = df_race['pct_overweight']

# Fit weighted linear regression using sample sizes as weights
model = sm.WLS(y, X, weights=df_race['Sample_Size'])
results = model.fit()

print(results.summary())

                            WLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.117
Model:                            WLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     268.2
Date:                Sun, 14 Dec 2025   Prob (F-statistic):           1.08e-56
Time:                        17:05:49   Log-Likelihood:                -6365.1
No. Observations:                2021   AIC:                         1.273e+04
Df Residuals:                    2019   BIC:                         1.275e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.2658      0.063    210.060      0.0

### Robustness Check

In [35]:
# different subsample - years 2014-2020
df_years = df_race[df_race['YearStart'] >= 2014]
df_years.head()

X = df_years['Indigenous']
X = sm.add_constant(X)
y = df_years['pct_overweight']

model = sm.WLS(y, X, weights=df_years['Sample_Size'])
results = model.fit()

print(results.summary())

                            WLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.104
Model:                            WLS   Adj. R-squared:                  0.103
Method:                 Least Squares   F-statistic:                     132.0
Date:                Sun, 14 Dec 2025   Prob (F-statistic):           5.63e-29
Time:                        17:06:07   Log-Likelihood:                -3534.8
No. Observations:                1137   AIC:                             7074.
Df Residuals:                    1135   BIC:                             7084.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.9738      0.083    156.262      0.0

## Model Evaluation and Diagnostics

In [ ]:
# checking for independence of residuals
residuals = results.resid

plt.figure(figsize=(8,6))
plt.plot(residuals)
plt.title('Residuals vs Observation from Weighted Linear Regression Model')
plt.xlabel('Observation')
plt.ylabel('Residuals')
plt.savefig('residuals_weighted_linear_regression.png')

In [112]:
# checking for homoskedasticity
fitted = results.fittedvalues

plt.figure(figsize=(8,6))
plt.scatter(fitted, residuals)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals vs Fitted Values')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.savefig('residuals_vs_fitted_weighted_linear_regression.png')

In [114]:
# checking for normality of residuals
sns.histplot(residuals, kde=True)
plt.title('Histogram of Residuals from Weighted Linear Regression Model')
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.savefig('histogram_residuals_weighted_linear_regression.png')

### Identifying Influential Points

In [21]:
# Cook's model for influential points
model = sm.OLS(y, X).fit()
influence = model.get_influence()
cooks_d, _ = influence.cooks_distance


# plot Cook's distance
plt.figure(figsize=(10, 6))
plt.stem(cooks_d)
plt.axhline(4/len(cooks_d), color='red', linestyle='--', label='Threshold: 4/n')
plt.title("Cook's Distance for Each Observation")
plt.ylabel("Cook's Distance")
plt.xlabel("Observation Index")
plt.legend()
plt.savefig('cooks_distance_weighted_linear_regression.png')


In [25]:
# identifying influential points from Cook's distance
threshold = 4/len(cooks_d)
influential_points = np.where(cooks_d > threshold)[0]
print("Influential points (indices):", influential_points)

Influential points (indices): [   0   34   36   40   99  102  120  122  147  158  183  202  204  212  223
  241  252  280  283  284  287  306  340  374  396  414  416  420  450  454
  482  488  496  504  506  516  536  552  576  580  583  585  613  631  684
  690  693  713  723  734  766  790  798  800  804  808  825  835  871  873
  874  883  900  984 1000 1025 1057 1082 1090 1092 1098 1126 1165 1166 1170
 1174 1274 1292 1369 1377 1384 1397 1401 1421 1425 1441 1443 1460 1555 1557
 1560 1577 1579 1631 1664 1671 1700 1842 1850 1858 1869 1900 1941 1967 1993
 2002 2012]


In [26]:
influential_points.shape

(107,)

In [ ]:
# create dataframe to display Cook's distance values
df_cooks = df_race.copy()
df_cooks['cooks_distance'] = cooks_d
df_cooks[df_cooks['cooks_distance'] > threshold]


,YearStart,YearEnd,LocationAbbr,LocationDesc,pct_overweight,Low_Confidence_Limit,High_Confidence_Limit,Sample_Size,age_months,race,GeoLocation,Indigenous,cooks_distance
3,2008,2008,AL,Alabama,25.0,19.3,30.7,228,NaN,Indigenous,"(32.84057112200048, -86.63186076199969)",1,0.002205
201,2008,2008,LA,Louisiana,27.0,20.3,33.7,174,NaN,Indigenous,"(31.31266064400046, -92.44568007099969)",1,0.003384
212,2008,2008,ME,Maine,27.4,20.1,34.7,146,NaN,Indigenous,"(45.254228894000505, -68.98503133599962)",1,0.003650
234,2008,2008,MA,Massachusetts,5.9,0.0,12.6,51,NaN,Indigenous,"(42.27687047000046, -72.08269067499964)",1,0.003621
581,2008,2008,PR,Puerto Rico,22.0,10.1,33.9,50,NaN,White,"(18.220833, -66.590149)",0,0.002276
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12268,2020,2020,CO,Colorado,7.3,5.2,10.2,411,NaN,Indigenous,"(38.843840757000464, -106.13361092099967)",1,0.002737
12460,2020,2020,MA,Massachusetts,25.0,15.2,38.2,52,NaN,Indigenous,"(42.27687047000046, -72.08269067499964)",1,0.002205
12628,2020,2020,OH,Ohio,7.4,4.6,11.7,215,NaN,Indigenous,"(40.06021014100048, -82.40426005599966)",1,0.002679
12688,2020,2020,SC,South Carolina,8.1,3.5,17.5,62,NaN,Indigenous,"(33.998821303000454, -81.04537120699968)",1,0.002288


## Model Comparison

In [ ]:
# removing influential points 
df_clean = df_race.drop(index=df_race.index[influential_points])

In [ ]:
# original dataframe
df_race.head()
df_race.shape

(2021, 12)

In [36]:
# cleaned dataframe without influential points
df_clean.head()
df_clean.shape

(1914, 12)

In [ ]:
# Fit weighted linear regression on cleaned dataframe
X_clean = df_clean[['Indigenous']]
X_clean = sm.add_constant(X_clean)
y_clean = df_clean['pct_overweight']
weights_clean = df_clean['Sample_Size']

model_clean = sm.WLS(y_clean, X_clean, weights=weights_clean).fit()
print(model_clean.summary())

                            WLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.094
Model:                            WLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     198.3
Date:                Sat, 06 Dec 2025   Prob (F-statistic):           6.44e-43
Time:                        17:01:46   Log-Likelihood:                -5880.4
No. Observations:                1914   AIC:                         1.176e+04
Df Residuals:                    1912   BIC:                         1.178e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.2407      0.062    214.790      0.0

In [161]:
df_race.to_csv('cleaned_wic_data.csv', index=False)

### Feature Importance

In [37]:
X_clean.head()

,const,Indigenous
9,1.0,0
14,1.0,1
20,1.0,0
25,1.0,1
31,1.0,0


In [38]:
X_numeric = X_clean.drop(columns='const')
standardized_X = (X_numeric - X_numeric.mean()) / X_numeric.std()
y_standardized = (y_clean - y_clean.mean()) / y_clean.std()

model_std = sm.WLS(y_standardized, sm.add_constant(standardized_X), weights=weights_clean).fit()
print(model_std.summary())

                            WLS Regression Results                            
Dep. Variable:         pct_overweight   R-squared:                       0.094
Model:                            WLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     198.3
Date:                Sat, 06 Dec 2025   Prob (F-statistic):           6.44e-43
Time:                        17:33:11   Log-Likelihood:                -3267.4
No. Observations:                1914   AIC:                             6539.
Df Residuals:                    1912   BIC:                             6550.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1874      0.036      5.147      0.0

In [39]:
coef_df = pd.DataFrame({
    'feature': X_numeric.columns,
    'coefficient': model_std.params[1:]  # exclude the constant term
}).sort_values("coefficient", ascending=False)

plt.figure(figsize=(8,5))
plt.bar(coef_df['feature'], coef_df['coefficient'])
plt.xticks(rotation=45)
plt.ylabel('Standardized Coefficient')
plt.title('Feature Importance based on Standardized Coefficients')
plt.savefig('feature_importance_standardized_coefficients.png')

## Result Visualizations

In [ ]:
# coefficient plot with confidence intervals
coefficient = results.params
confidence_intervals = results.conf_int()
confidence_intervals.columns = ['lower', 'upper']

df_ci_plot = pd.DataFrame({
    'coefficient': coefficient,
    'lower': confidence_intervals['lower'],
    'upper': confidence_intervals['upper']
})


In [41]:
# plotting coefficients with error bars
plt.figure(figsize=(8,5))

plt.errorbar(
    df_ci_plot['coefficient'],
    df_ci_plot.index,
    xerr=[df_ci_plot['coefficient'] - df_ci_plot['lower'],
          df_ci_plot['upper'] - df_ci_plot['coefficient']],
    fmt='o',
    capsize=4
)

plt.axvline(0, color='red', linestyle='--')
plt.title("Coefficient Plot with 95% Confidence Intervals")
plt.xlabel("Coefficient Value")
plt.ylabel("Features")
plt.savefig('coefficient_plot_confidence_intervals.png')

In [ ]:
# plotting Boxplot of the relationship between Indigenous vs White WIC Participants and % Overweight
plt.figure()
sns.boxplot(x='race', y='pct_overweight', data=df_race)
plt.title("Boxplot of Indigenous vs White WIC Participants and % Overweight")
plt.ylabel('% of WIC toddlers overweight')
plt.savefig('boxplot_indigenous_pct_overweight.png')

In [45]:
# plotting time series of % overweight over years for Indigenous and White WIC Participants
plt.figure(figsize=(10,6))
sns.lineplot(x='YearStart', y='pct_overweight', hue='race', data=df_race, marker='o')
plt.title('% Overweight over Years for Indigenous and White WIC Participants')
plt.xlabel('Year')
plt.ylabel('% of WIC toddlers overweight')
plt.savefig('time_series_pct_overweight_years.png')